<a href="https://colab.research.google.com/github/leahgriffith123/SN-Classifier/blob/master/Edge_TPU_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing the Model

In [ ]:
from astropy.visualization import ZScaleInterval
import datetime
from IPython.display import display
import matplotlib.pyplot as plt
from multiprocessing.pool import ThreadPool
import numpy as np
import pathlib
from sklearn.metrics import roc_curve, auc, confusion_matrix, multilabel_confusion_matrix
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_addons as tfa
from tqdm.auto import tqdm

!git clone https://github.com/dmitryduev/ay119.git

Cloning into 'ay119'...
remote: Enumerating objects: 337, done.
remote: Counting objects: 100% (337/337), done.
remote: Compressing objects: 100% (317/317), done.
remote: Total 337 (delta 21), reused 330 (delta 17), pack-reused 0
Receiving objects: 100% (337/337), 16.53 MiB | 8.36 MiB/s, done.
Resolving deltas: 100% (21/21), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
path_base = '/content/'
path_data = '/content/ay119/deep_learning/data'

leah_path_data = '/content/drive/My Drive/Leah_SURF/Data'

In [ ]:
p = pathlib.Path(leah_path_data)
path_model = p / 'models' / 'tl_3c_v1.20200803_155635.h5'
model_tl = tf.keras.models.load_model(path_model)
model_tl.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1_input (InputLayer)     [(None, 63, 63, 3)]       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 61, 61, 16)        448       
_________________________________________________________________
conv2 (Conv2D)               (None, 59, 59, 16)        2320      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 29, 29, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 29, 29, 16)        0         
_________________________________________________________________
conv3 (Conv2D)               (None, 27, 27, 32)        4640      
_________________________________________________________________
conv4 (Conv2D)               (None, 25, 25, 32)        9248

# Data Download

In [ ]:
def load_helper(pp):
    data = np.load(pp)
    for i in range(3):
        data[..., i] /= np.linalg.norm(data[..., i])
    return data

p = pathlib.Path(leah_path_data)

n_bogus = len(list((p / 'bogus').glob('*.npy')))
n_realsn = len(list((p / 'realsn').glob('*.npy')))
n_realagn = len(list((p / 'realagn').glob('*.npy')))
n_bogus, n_realsn, n_realagn

(3700, 3762, 3772)

In [ ]:
with ThreadPool(processes=12) as pool:
    bogus = list(tqdm(pool.imap(load_helper, (p / 'bogus').glob('*.npy')), total=n_bogus))

In [ ]:
with ThreadPool(processes=12) as pool:
    realsn = list(tqdm(pool.imap(load_helper, (p / 'realsn').glob('*.npy')), total=n_realsn))

In [ ]:
with ThreadPool(processes=12) as pool:
    realagn = list(tqdm(pool.imap(load_helper, (p / 'realagn').glob('*.npy')), total=n_realagn))

In [ ]:
bogus = np.array(bogus[0:100])
realsn = np.array(realsn[0:100])
realagn = np.array(realagn[0:100])

In [ ]:
data = np.vstack((bogus, realsn, realagn))

In [ ]:
indexes = [0 for _ in range(bogus.shape[0])] +\
    [1 for _ in range(realsn.shape[0])] +\
    [2 for _ in range(realagn.shape[0])]
labels = tf.one_hot(indexes, depth=3, dtype=tf.int8).numpy()
labels.shape

(300, 3)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((data, labels))

# Edge TPU Conversion


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_tl)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
def representative_dataset_gen():
  for _ in range(len(dataset)):
    # Get sample input data as a numpy array in a method of your choosing.
    input = next(iter(dataset))
    yield [input]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
tflite_quant_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpjsxplsvo/assets


INFO:tensorflow:Assets written to: /tmp/tmpjsxplsvo/assets


AttributeError: ignored